In [128]:
import pandas as pd
import numpy as np
import os

In [129]:
pwd

'C:\\Users\\consultant137\\Desktop\\Sanjeev Folder'

In [130]:
import dask.dataframe as dd
import glob

### Reading comments file

In [131]:
df1 = pd.read_csv('Schonfield\meo_df_879.csv')

In [134]:
 days = df1['ViewData.Task Business Date'].value_counts().reset_index()

In [136]:
date = list(days[days['ViewData.Task Business Date']>50]['index'])[0]

In [138]:
date1 = pd.to_datetime(date)

In [143]:
day = date1.day
mon = date1.month
yr = date1.year

In [174]:
path = 'Schonfield\daily files\\'

In [175]:
filename = '57 Position recon - '+ str(mon) + str(day) + str(yr)+'.csv'

In [176]:
filename1 = path +filename

In [177]:
filename1

'Schonfield\\daily files\\57 Position recon - 9222020.csv'

In [178]:

pos = pd.read_csv(filename1)

FileNotFoundError: [Errno 2] File b'Schonfield\\daily files\\57 Position recon - 9222020.csv' does not exist: b'Schonfield\\daily files\\57 Position recon - 9222020.csv'

In [51]:
rem_word = ['comdty','index','indx','elec']

In [52]:
def tickerclean(x):
    if ((x!= None) & (type(x)== str)):
        x = x.lower()
        x1 = x.split()
        k = []
        for item in x1:
            if item not in rem_word:
                k.append(item)
        return ' '.join(k)

In [53]:
pos = pos[['Fund', 'Custodian Account','Investment ID', 'Strategy', 'Security Type', 'Currency',
        'Description', 'Quantity Diff','Local Price Diff', 'Price dif %','Local MV Diff']]

In [54]:
pos = pos.rename(columns = {'Description':'Pos_Desc','Security Type':'Pos_security','Quantity Diff':'pos_qnt_diff','Investment ID':'Ticker'})

In [55]:
pos['Ticker1'] = pos['Ticker'].apply(lambda x : tickerclean(x) )

In [57]:
df1 = df1.rename(columns = {'ViewData.Mapped Custodian Account':'Custodian Account',
                           'ViewData.Currency':'Currency',
                            'ViewData.Ticker':'Ticker',
                          'ViewData.Net Amount Difference':'Net Amount Difference',
                           'ViewData.Settle Date':'Settle Date',
                           'ViewData.Trade Date':'Trade Date',
                           'ViewData.Description':'Description'})

In [58]:
df1['Ticker1'] = df1['Ticker'].apply(lambda x : tickerclean(x))

In [61]:
df1 = pd.merge(df1, pos, on = ['Custodian Account','Currency','Ticker1'], how = 'left')

In [63]:
df1 = df1.reset_index()

In [64]:
df1 =df1.drop('index',1)

### Cleaning of Variables and feature Engineering

- cleaning of dates

In [65]:
df2= df1.copy()

In [66]:
import datetime

In [67]:
df2['Settle Date'] = pd.to_datetime(df2['Settle Date'])
df2['Trade Date'] = pd.to_datetime(df2['Trade Date'])
#df2['ViewData.Task Business Date'] = pd.to_datetime(df2['ViewData.Task Business Date'])

In [68]:
#df2['ViewData.Task Business Date1'] = df2['ViewData.Task Business Date'].dt.date

In [69]:
df2['Settle Date1'] = df2['Settle Date'].dt.date
df2['Trade Date1'] = df2['Trade Date'].dt.date

- Cleaning of description

In [70]:
def desc_clean(x):
    if type(x)== str:
        x = x.lower()
        x1 = x.split()
        if 'interest' in x1:
            return 'interest'
        elif (('fx' in x1) & ('conv' in x1)):
            return 'fx conv'
        elif (('comm' in x1) & ('adj' in x1)):
            return 'commission'
        elif(('fee' in x1) & ('adj' in x1)):
            return 'fee'
        elif (('aps' in x1) & ('adj' in x1)):
            return 'aps'
        elif (('internal' in x1) & ('wire' in x1) & ('transfer' in x1)):
            return 'internal wire transfer'
        elif (('incoming' in x1) & ('fedwire(s)' in x1)):
            return 'incoming fedwire'
        else:
            return x
        

In [71]:
df2['Description'] = df2['Description'].apply(lambda x :desc_clean(x))

- Taking care of the amount variable

In [217]:
pd.set_option('display.max_columns', 500)

In [303]:
df2.head(3)

,ViewData.Account Name,ViewData.Account Type,ViewData.Accounting Base Net Amount,ViewData.Accounting Cancel Amount,ViewData.Accounting Fund,ViewData.Accounting Net Amount,ViewData.Accounting Quantity,ViewData.Activity Code,ViewData.Age,ViewData.Age WK,ViewData.Alt ID 1,ViewData.Asset Type Category,ViewData.Assigned To,ViewData.Balance Derived Source,ViewData.Balance FX Rate,ViewData.BalanceAcc,ViewData.BalancePB,ViewData.Balances Account Name,ViewData.Balances Alt ID 1,ViewData.Balances Balance FX Rate,ViewData.Balances Fund,ViewData.Balances Fund Structure,ViewData.Balances PB Account Number,ViewData.Base Closing Balance,ViewData.Base Currency,ViewData.Base Net Amount,ViewData.Base Net Amount Difference,ViewData.BaseBalanceAcc,ViewData.BaseBalancePB,ViewData.Bloomberg_Yellow_Key,ViewData.Break Tag,ViewData.BreakID,ViewData.Business Date,ViewData.CUSIP,ViewData.Call Put Indicator,ViewData.Cancel Amount,ViewData.Cancel Amount Difference,ViewData.Cancel Flag,ViewData.Client,ViewData.ClientShortCode,ViewData.Closing Balance,ViewData.Closing Date,ViewData.ClusterID,ViewData.CombinedAndIsPaired,ViewData.Commission,Currency,ViewData.Cust Base Net Amount,ViewData.Cust Cancel Amount,ViewData.Cust Fund,ViewData.Cust Net Amount,ViewData.Cust Quantity,ViewData.Custodian,ViewData.Custodian Account,ViewData.Custom 1,ViewData.Custom 10,ViewData.Custom 11,ViewData.Custom 12,ViewData.Custom 13,ViewData.Custom 14,ViewData.Custom 15,ViewData.Custom 16,ViewData.Custom 17,ViewData.Custom 18,ViewData.Custom 19,ViewData.Custom 2,ViewData.Custom 20,ViewData.Custom 3,ViewData.Custom 4,ViewData.Custom 40,ViewData.Custom 41,ViewData.Custom 5,ViewData.Custom 6,ViewData.Custom 7,ViewData.Custom 8,ViewData.Custom 9,ViewData.Department,ViewData.Derived Source,ViewData.Description,ViewData.ETL File Code,ViewData.ETL Package Code,ViewData.ExpiryDate,ViewData.ExternalComment1,ViewData.ExternalComment2,ViewData.ExternalComment3,ViewData.FX Rate,ViewData.Fund,ViewData.Fund Difference,ViewData.Fund Structure,ViewData.Group,ViewData.Has Attachments,ViewData.ISIN,ViewData.Interest Amount,ViewData.Interest Balance,ViewData.InternalComment1,ViewData.InternalComment2,ViewData.InternalComment3,ViewData.Investment ID,ViewData.Investment Type,ViewData.Is Combined Data,ViewData.Keys,ViewData.Knowledge Date,ViewData.Legal Entity,ViewData.LoanX ID,Custodian Account,Net Amount Difference,ViewData.Net Amount Difference Absolute,ViewData.Non Trade Description,ViewData.OTE Custodian Account,ViewData.OTE Ticker,ViewData.OTEIncludeFlag,ViewData.PB Account,ViewData.PB Account Number,ViewData.PB Account Numeric,ViewData.Portfolio ID,ViewData.Portolio,ViewData.Price,ViewData.Prime Broker,ViewData.Principal Amount,ViewData.Principal Balance,ViewData.Prior Knowledge Date,ViewData.Priority,ViewData.Quantity,ViewData.Quantity Difference,ViewData.Recon Data Model,ViewData.Recon Purpose,ViewData.Recon Setup,ViewData.Recon Setup Code,ViewData.Reviewer,ViewData.Rule And Key,ViewData.SEDOL,ViewData.SPM ID,ViewData.Sec Fees,ViewData.Settle Date,ViewData.Side0_UniqueIds,ViewData.Side1_UniqueIds,ViewData.Source,ViewData.Source Combination,ViewData.Source Combination Code,ViewData.Source Type Code Combination,ViewData.Source Type Combination,ViewData.SourceTypeCode,ViewData.Status,ViewData.Strategy,ViewData.Strike Price,ViewData.System Comments,ViewData.TabID,ViewData.Task Business Date,ViewData.Task ID,ViewData.Task Knowledge Date,Ticker_x,ViewData.Trade Date,ViewData.Trade Expenses,ViewData.Transaction Category,ViewData.Transaction ID,ViewData.Transaction Type,ViewData.Type,ViewData.UnMapped,ViewData.Underlying Cusip,ViewData.Underlying ISIN,ViewData.Underlying Investment ID,ViewData.Underlying Sedol,ViewData.Underlying Ticker,ViewData.UserTran1,ViewData.UserTran2,ViewData.Value Date,ViewData.Workflow Remark,ViewData.Workflow Status,ViewData._Actions,ViewData._GroupID,ViewData._ID,_id,Ticker1,Fund,Ticker_y,Strategy,Pos_security,Pos_Desc,pos_qnt_diff,Local Price Diff,Price dif %,Local MV Diff,

In [72]:
def amountcleaning(x):
    if type(x)==str:
        
        if x.startswith('('):
            x1 = x.strip('(,)')
            x2 = x1.split(',')
            x3 = []
            for item in x2:
                if item!=',':
                    x3.append(item)
            x4 = ''.join(x3)
                
            x4 = 0 - float(x4)
            return x4
        else:
            x2 = x.split(',')
            x3 = []
            for item in x2:
                if item!=',':
                    x3.append(item)
            x4 = ''.join(x3)
            return float(x4)
    else:
        return 1234567

In [74]:
df2['Net Amount Difference1']=df2['Net Amount Difference']

In [75]:
#df2['Net Amount Difference1'] = df2['Net Amount Difference'].apply(lambda x : amountcleaning(x))
df2['Local MV Diff'] = df2['Local MV Diff'].apply(lambda x :amountcleaning(x) )

In [76]:
dummyk = df2.groupby(['Custodian Account','Currency','Ticker1'])['Net Amount Difference1'].sum().reset_index()

In [79]:
dummyk = dummyk.rename(columns = {'Net Amount Difference1':'Cash Standing'})

In [80]:
df2 = pd.merge(df2,dummyk, on = ['Custodian Account','Currency','Ticker1'], how = 'left')

In [81]:
df2['cash difference'] = df2.apply(lambda x : x['Cash Standing']+x['Local MV Diff'] if x['Local MV Diff']!=1234567.0 else 1234567, axis =1 )

### Elimination of Matched

#### Absolute Zero : No tolerance

In [82]:
dummy = df2.groupby(['Custodian Account','Currency','Ticker1'])['Net Amount Difference1'].apply(list).reset_index()

In [84]:
def subSum(numbers,total):
    length = len(numbers)
    
    
    
    if length <20:
      
            
      
        
        for index,number in enumerate(numbers):
            if np.isclose(number, total, atol=5).any():
                return [number]
                print(34567)
            subset = subSum(numbers[index+1:],total-number)
            if subset:
                #print(12345)
                return [number] + subset
        return []
    else:
        return numbers

In [89]:
dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))

In [90]:
dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))

In [91]:
dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))

In [92]:
dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']

In [93]:
dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))

In [321]:
#dummy['remain_amt'] = dummy.apply(lambda x : list(set(x['Net Amount Difference1'])-set(x['zero_list'])) if x['remove_mark'] == 1 else "AA", axis =1)

In [95]:
dummy = pd.merge(dummy, pos , on = ['Custodian Account','Currency','Ticker1'], how = 'left')

In [96]:
def remove_mark(x,y,z,k):
    
   
    if ((x>1) & (x<20)):
        if ((k<6.0) & (z==0) & (y=='0.00')):
            return 1
        elif ((k==0.0) & (z!=0) & (y=='0.00')):
            return 1
        else:
            return 0
    else:
        return 0

In [97]:
dummy['remove_mark'] = dummy.apply(lambda x :remove_mark(x['zero_list_len'],x['pos_qnt_diff'],x['diff_len'],x['zero_list_sum']),axis = 1)

In [99]:
dummy = dummy[['Custodian Account', 'Currency', 'Ticker1', 'zero_list',  'diff_len', 'remove_mark']]

In [100]:
df3 = pd.merge(df2, dummy, on = ['Custodian Account','Currency','Ticker1'], how = 'left')

In [102]:
def remover(x,y,z):
    if x==1:
        if y in z:
            return 1
        else:
            return 0
    else:
        return 0
   

In [103]:
df3['remove_mark_fin'] = df3.apply(lambda x : remover(x['remove_mark'],x['Net Amount Difference1'],x['zero_list']),axis =1)

In [113]:
df4 = df3[df3['remove_mark_fin']==1]
df5 = df3[df3['remove_mark_fin']!=1]

In [115]:
if df4.shape[0] !=0:
    df4['Predicted Comment'] = 'Match'
    df4.to_csv('Schonfield 897 Meo Prediction P1.csv')
else:
    df5 = df5.copy()

C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [335]:
df5.drop(['zero_list', 'diff_len', 'remove_mark',
       'remove_mark_fin'], axis =1 , inplace = True)

In [336]:
dummy = df5.groupby(['Custodian Account','Currency'])['Net Amount Difference1'].apply(list).reset_index()

In [337]:
dummy['len_amount'] = dummy['Net Amount Difference1'].apply(lambda x : len(x))

In [338]:
dummy['zero_list'] = dummy['Net Amount Difference1'].apply(lambda x : subSum(x,0))

In [339]:
dummy['zero_list_len'] = dummy['zero_list'].apply(lambda x : len(x))

In [340]:
dummy['diff_len'] = dummy['len_amount'] - dummy['zero_list_len']

In [341]:
dummy['zero_list_sum'] = dummy['zero_list'].apply(lambda x : round(abs(sum(x)),2))

In [343]:
def remove_mark_next(x,z,k):
    
   
    if ((x>1) & (x<20)):
        if ((k<6.0) & (z==0)):
            return 1
        elif ((k==0.0) & (z!=0)):
            return 1
        else:
            return 0
    else:
        return 0

In [344]:
dummy['remove_mark'] = dummy.apply(lambda x :remove_mark_next(x['zero_list_len'],x['diff_len'],x['zero_list_sum']),axis = 1)

In [345]:
dummy = dummy[['Custodian Account', 'Currency', 'zero_list',  'diff_len', 'remove_mark']]

In [346]:
df3 = pd.merge(df5, dummy, on = ['Custodian Account','Currency'], how = 'left')

In [347]:
df3['remove_mark_fin'] = df3.apply(lambda x : remover(x['remove_mark'],x['Net Amount Difference1'],x['zero_list']),axis =1)

In [350]:
df4 = df3[df3['remove_mark_fin']==1]
df5 = df3[df3['remove_mark_fin']!=1]

In [ ]:
if df4.shape[0] !=0:
    df4['Predicted Comment'] = 'Match'
    df4.to_csv('Schonfield 897 Meo Prediction P2.csv')
else:
    df5 = df5.copy()

In [110]:
df6 = df5[df5['ViewData.InternalComment2'].isna()]
df7 = df5[~df5['ViewData.InternalComment2'].isna()]

In [116]:
if df7.shape[0]!=0:
    df7['Predicted Comment'] = df7['ViewData.InternalComment2']
else:
    df6 = df6.copy()

In [117]:
def commentschon(pos,amt,accamt, pbamt,cash_diff):
    if ((pos=='0.00') & (amt=='0.00')):
        if((cash_diff<6.0) & (cash_diff>-6.0)):
            com = 'MV Swing'
        else:
            
            com = 'Commission & fee difference,SFA to advise'
    elif(pos!='0.00'):
        if (accamt==None):
            com = 'GVA missing the trade, viteos to check and book'
        else:
            com = 'PB to report missing the trade.'
    else:
        com = 'MV Swing'
        
    return com
            

In [118]:
df6['Predicted Comment'] = df6.apply(lambda x : commentschon(x['pos_qnt_diff'],x['Local Price Diff'],x['ViewData.Accounting Net Amount'],x['ViewData.Cust Net Amount'],x['cash difference']),axis = 1)

In [120]:
df7.to_csv('Schonfield 897 Meo Prediction P3.csv')
df6.to_csv('Schonfield 897 Meo Prediction P4.csv')

#### Combining all the files

In [121]:
a = pd.read_csv('Schonfield 897 Meo Prediction P1.csv')
b = pd.read_csv('Schonfield 897 Meo Prediction P2.csv')
c = pd.read_csv('Schonfield 897 Meo Prediction P3.csv')
d = pd.read_csv('Schonfield 897 Meo Prediction P4.csv')

In [122]:
frames = [a,b,c,d]

In [123]:
final_df = pd.concat(frames)

C:\Users\consultant137\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [126]:
final_df = final_df.rename(columns = {'Custodian Account':'ViewData.Mapped Custodian Account',
                           'Currency':'ViewData.Currency',
                            'Ticker':'ViewData.Ticker',
                          'Net Amount Difference':'ViewData.Net Amount Difference',
                          'Settle Date': 'ViewData.Settle Date',
                           'Trade Date':'ViewData.Trade Date',
                           'Description':'ViewData.Description'})

In [127]:
final_df.to_csv('Prediction on Daily Meo of Schonfield 897.csv')